In [6]:
# Install dependencies with pip (needed for Codespaces)
%pip install numpy
%pip install qiskit
%pip install qiskit_ibm_runtime

import sys
import math
import base64
import random
import hashlib
import time
import numpy as np
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Operator
from numpy import pi

214.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


220.09s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


226.12s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [7]:
# Author: David Reese
# Simple Python RSA Encryption Algorithm

# Increase the recursion limit for deep recursive functions
sys.setrecursionlimit(1500)

# Greatest Common Denominator
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


# check if p is prime
# do the test k times
def fermat(p, k=2):
    # base case
    if p < 2:
        return False
    # quick check for small primes & common divisors
    if p in [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]:
        return True
    if p % 2 == 0 or p % 3 == 0 or p % 5 == 0:
        return False
    # generate random number & test
    for _ in range(k):
        a = random.randint(2, p - 2)
        if pow(a, p - 1, p) != 1:
            return False
    # if the code reaches this point, unless it is a carmichael number,
    # we should have our result...
    return True


# divide-and-conquer algo for multiplying large numbers
# theoretically faster than FFT at O(n^1.58)
# STEPS:
# 1. split the input into two halves ( 7776 -> 77 and 76)
# 2. recursively multiply the low & high halves, and calculate product of sum of halves.
# 3. combine the final product.
def karatsuba(x, y):
    # base case:
    if x < 10 and y < 10:
        return x * y
    # first step
    len_x = len(str(x))
    len_y = len(str(y))
    # calculate midpoint m
    m = max(len_x, len_y) // 2
    a = x // 10 ** m
    b = x % 10 ** m
    c = y // 10 ** m
    d = y % 10 ** m
    # recursive steps
    ac = karatsuba(a, c)
    bd = karatsuba(b, d)
    product = karatsuba(a + b, c + d) - ac - bd
    # combine & return result
    return ac * 10 ** (2 * m) + (product * 10 ** m) + bd


# Generates e with a bias towards higher numbers
def generate_e(phi):
    # descending list of e candidates, likely 65537 will work
    candidates = [65537, 32767, 17011, 8191, 4093, 2039, 1021, 509, 257, 127, 61, 37, 17]
    for e in candidates:
        if gcd(e, phi) == 1:  # Is it co-prime?
            return e
    return None


def generate_rsa_keys():
    primes = []
    while len(primes) < 2:
        p = random.randint(2**15, 2**16)  # smaller primes for demonstration
        if fermat(p):
            primes.append(p)
    p, q = primes
    n = karatsuba(p, q)
    phi = karatsuba(p-1, q-1)
    e = generate_e(phi)
    d = pow(e, -1, phi)
    return {'p': p, 'q': q, 'n': n, 'phi': phi, 'e': e, 'd': d}


# Generate RSA keys
keys = generate_rsa_keys()
print("Generated RSA keys:", keys)

Generated RSA keys: {'p': 36899, 'q': 49531, 'n': 1827644369, 'phi': 1827557940, 'e': 65537, 'd': 535018793}


In [8]:
# Qiskit Shor's Algorithm Program

# Import necessary modules from Qiskit
import os
from qiskit import QuantumCircuit
from math import pi, log, ceil, sqrt

# Define a function for classical preprocessing of the number N
def classical_preprocessing(N):
    # Check if N is even
    if N % 2 == 0:
        return f"{N} is even, no need for quantum algorithm."
    
    # Check if N is a power of a prime
    for base in range(2, int(sqrt(N)) + 1):
        power = round(log(N, base))
        if base ** power == N:
            return f"{N} is a power of {base}^({power}), no need for quantum algorithm."
    
    # If none of the above, return None to proceed with quantum computation
    return None

# Define a controlled unitary function for the modular multiplication
def c_amod15(a, power, n):
    """Controlled multiplication by a modulo 15, raised to the power of 2^power."""
    total_qubits = n + 1  # Total qubits includes the control qubit
    U = QuantumCircuit(total_qubits)
    for _ in range(power):
        # Perform the modular multiplication operation on the qubits
        for q in range(n):
            U.swap(q, (q + 1) % n)
        U.x(range(n))  # Apply a NOT gate to all operational qubits
    U_gate = U.to_gate()
    c_U = U_gate.control(1)  # Control by one additional qubit
    return c_U

# Define the inverse Quantum Fourier Transform (QFT†)
def qft_dagger(n):
    """n-qubit QFTdagger on the first n qubits in a circuit."""
    qc = QuantumCircuit(n)
    # Apply the inverse QFT
    for j in range(n):
        for k in range(j):
            # Apply controlled phase gates
            qc.cp(-pi/float(2**(j-k)), k, j)
        # Apply Hadamard gate to each qubit
        qc.h(j)
    return qc

# Define the quantum order finding circuit
def order_finding_circuit(N, a):
    n = ceil(log(N, 2))
    total_qubits = 2 * n + 1
    qc = QuantumCircuit(total_qubits, n)
    qc.h(range(n))  # Initialize in superposition
    for q in range(n):
        qc.append(c_amod15(a, 2**q, n), [q] + list(range(n, 2*n)) + [2*n])
    qc.append(qft_dagger(n), range(n))
    qc.measure(range(n), range(n))
    return qc

# Set the number to be factored and the base for the order finding
N = keys['n']  # Use the RSA encryption key N as the number to be factored
a = 7          # Base number for order finding

# Perform classical preprocessing and decide whether to proceed with quantum steps
preprocessing_result = classical_preprocessing(N)
if preprocessing_result:
    print(preprocessing_result)
else:
    # Construct the quantum circuit if classical preprocessing does not resolve the problem
    qc = order_finding_circuit(N, a)
    # Draw the quantum circuit using the 'matplotlib' drawer
    qc.draw(output='mpl')

    # Quantum IBM Connection for Running on Real Hardware
    IBM_TOKEN = os.environ.get('IBM_QUANTUM_TOKEN')

    # Authenticate with IBM Quantum
    service = QiskitRuntimeService(channel="ibm_quantum", token=os.getenv('IBM_QUANTUM_TOKEN'))

    # Choose a backend
    backend = service.backend("ibmq_qasm_simulator")

    # Create a Sampler job for the quantum circuit
    sampler = Sampler(circuits=qc, service=service, backend=backend)

    # Run the Sampler job
    job = sampler.run()

    # Wait for the job to complete and fetch the results
    result = job.result()

    # Print the results
    print("Counts from the circuit execution:", result)
    